This Notebook is a brief sketch of how to use the Deutsch-Jozsa algorithm.
We start by declaring all necessary imports.

In [1]:
from collections import defaultdict
from mock import patch, Mock
from itertools import product

from pyquil.api import SyncConnection
from pyquil.job_results import JobResult, wait_for_job
import numpy as np

from grove.deutsch_jozsa.deutsch_jozsa import DeutschJosza

The Deutsch-Jozsa Algorithm can be used to determine if a binary-valued function is constant or balanced (returns 0 on exactly half of its inputs).

In [2]:
bit_value = '0'
bit = ("0", "1")
constant_bitmap = {}

# We construct the bitmap for the algorithm
for bitstring in product(bit, repeat=2):
    bitstring = "".join(bitstring)
    constant_bitmap[bitstring] = bit_value

We verify that we have a constant bitmap below.

In [3]:
for value in constant_bitmap.values():
    assert value == bit_value, "The constant_bitmap is not constant with value bit_value."

To use the Deutsch-Jozsa algorithm on a Quantum Hardware we need to define the connection to the QVM or QPU. However we don't have a real connection in this notebook, so we just mock out the response. If you run this notebook, ensure to replace `cxn` with a pyQuil connection object.

In [4]:
with patch("pyquil.api.SyncConnection") as cxn:
    # Need to mock multiple returns as an iterable
    mock_job_result = Mock(spec=JobResult)
    mock_job_result.is_done.return_value = True
    mock_job_result.result = {'result': [[0, 0]]}
    cxn.run_and_measure.return_value = mock_job_result

Now let's run the Deutsch Jozsa algorithm. We instantiate the Deutsch Jozsa object and then call its `is_constant` method with the connection object and the bitmap we defined above. Finally we assert its correctness by checking the output. (The method returns a boolean, so here we just check the returned boolean.)

In [5]:
dj = DeutschJosza()
is_constant = dj.is_constant(cxn, constant_bitmap)
assert is_constant, "The algorithm said the function was balanced."

If the assertion succeeded we know the algorithm returned the zero bitstring and successfully recognized the function as constant.